In [ ]:
# 测试prompt-bert
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

# 使用 'bert-base-uncased' 模型
model_name = 'result/28/'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 示例句子
sentence1 = "I love machine learning and natural language processing."
# sentence2 = "I hate machine learning and natural language processing."
template = 'This sentence : "{sentence}" means [MASK].'
mask_token_id = tokenizer.mask_token_id

sentence2 = template.format(sentence=sentence1)


# 将句子转换为token ID，并添加特殊token [CLS] 和 [SEP]
inputs1 = tokenizer(sentence1, return_tensors='pt', padding=True, truncation=True)
inputs2 = tokenizer(sentence2, return_tensors='pt', padding=True, truncation=True)

# 模型不需要计算梯度，因此使用 torch.no_grad()
with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

# BERT 输出的是一个包含多层的输出，这里我们只关心最后一层的隐藏状态
last_hidden_state1 = outputs1.last_hidden_state
last_hidden_state2 = outputs2.last_hidden_state

# 取 [CLS] token 对应的向量，作为整个句子的向量表示
sentence_embedding1 = last_hidden_state1[:, 0, :]  # [batch_size, hidden_size]
# sentence2_embedding = last_hidden_state2[:, 0, :]  # [batch_size, hidden_size]
sentence2_embedding = last_hidden_state2[inputs2['input_ids'] == mask_token_id]  # [batch_size, hidden_size]
# sentence2_embedding = sentence2_embedding.view(1, -1)

cos_sim = F.cosine_similarity(sentence_embedding1, sentence2_embedding)

print(cos_sim.item())


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

model_name = 'princeton-nlp/unsup-simcse-bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 示例句子
sentence = "I love machine learning and natural language processing."

# 通过不同的方式计算句子表征
# 1 什么都不做，直接使用 [CLS] token 的向量
inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    outputs1 = model(**inputs)
    emb1 = outputs1.last_hidden_state[:, 0, :]
cos_sim = F.cosine_similarity(emb1, emb1)
print("余弦相似度 (直接使用[CLS] token 向量):", cos_sim.item())

# 2 整个句子进去
template = 'This sentence : "{sentence}" means [MASK].'
prompt_sentenct = template.format(sentence=sentence).replace('[MASK]', tokenizer.mask_token)
inputs2 = tokenizer(prompt_sentenct, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    outputs2 = model(**inputs2)
    emb2 = outputs2.last_hidden_state[:, 0, :] # cls
    emb3 = outputs2.last_hidden_state[inputs2['input_ids'] == tokenizer.mask_token_id]  # mask
cos_sim = F.cosine_similarity(emb1, emb2)
cos_sim2 = F.cosine_similarity(emb1, emb3)
print("余弦相似度 (使用Prompt-BERT的整个句子向量):", cos_sim.item())
print("余弦相似度 (使用Prompt-BERT的[MASK] token 向量):", cos_sim2.item())



In [ ]:
# 3 引入attention mask
template = 'This sentence : "{sentence}" means [MASK].'.replace('[MASK]', tokenizer.mask_token)
prefix = template.split("{sentence}")[0]
suffix = template.split("{sentence}")[1]
prefix_input_ids = tokenizer(prefix, return_tensors='pt', padding=True, truncation=True)['input_ids'].view(-1)
suffix_inputs_ids = tokenizer(suffix, return_tensors='pt', padding=True, truncation=True)['input_ids'].view(-1)
sentence_inputs_ids = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)['input_ids'].view(-1)

prefix_input_ids = prefix_input_ids[:-1]
suffix_inputs_ids = suffix_inputs_ids[1:]
sentence_inputs_ids = sentence_inputs_ids
input_ids = torch.cat([prefix_input_ids, sentence_inputs_ids, suffix_inputs_ids])

prompt_weight = 0
attention_mask = torch.cat([
    torch.full(prefix_input_ids.size(),prompt_weight),
    torch.full(sentence_inputs_ids.size(),float(1)),
    torch.full(suffix_inputs_ids.size(),prompt_weight)
])

inputs3 = {'input_ids': input_ids.view(1, -1), 'attention_mask': attention_mask.view(1, -1)}
with torch.no_grad():
    outputs3 = model(**inputs3)
    emb3 = outputs3.last_hidden_state[inputs3['input_ids'] == tokenizer.mask_token_id]  # mask
cos_sim3 = F.cosine_similarity(emb1, emb3)
print("引入attention mask的的相似度", cos_sim3.item())

In [ ]:
# 采样数据集
import json
import random
import os
import numpy as np

dataset_path = 'data/wiki1m_for_simcse_ner.json'

with open(dataset_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

print("数据集大小:", len(dataset))
print("数据集示例:", dataset[0])

# 采样1000个样本
dataset_sample = random.sample(dataset, 10)


In [ ]:
dict_path = 'data/wiki1m_for_simcse_ner_entity_dict.json'
with open(dict_path, 'r', encoding='utf-8') as f:
    entity_dict = json.load(f)


In [ ]:
import hashlib
def hash(text):
    return hashlib.md5(text.encode()).hexdigest()

output_path = 'data/wiki1k_with_entity_knowledge.json'
for item in dataset_sample:
    entity_list = item['entities']
    print(item)
    for entity in entity_list:
        entity_hash = hash(entity['text']) + '.json'
        if entity_hash in entity_dict:
            entity['knowledge'] = entity_dict[entity_hash]
            print(entity)
    break

In [ ]:
# 测试apex是否可用
from transformers.utils import is_apex_available
print(is_apex_available())

In [ ]:
# 测试评测能力
from utils.auto_eval import eval

eval('model/unsup-PromptBERT-baseline',pooler='avg')

In [ ]:
# 测试bert attention mask
import torch
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F

model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

sentence = "I love machine learning and natural language processing."

inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)


inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])

padding = torch.full((inputs['input_ids'].shape[0], 5), fill_value=0, dtype=inputs['input_ids'].dtype)

output = model(input_ids=padding)
last_hidden_state1 = output.last_hidden_state[:, 0, :]
pass


In [ ]:
import nltk
nltk.download('punkt_tab')

In [1]:
# 检索测试
from knowledge.retrieval import retrieve_knowledge
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import torch
import redis
import base64
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import json

def text_encode(text):
    # base64 编码
    return base64.b64encode(text.encode()).decode()

input_file = 'data/wiki1m_for_simcse.txt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'princeton-nlp/unsup-simcse-bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).cuda()

r = redis.Redis(host='59.77.134.205', port=6379, db=2, password='lyuredis579')
prefix = 'similarity_sent_'
topk = 10

with open(input_file, 'r', encoding='utf-8') as f:
    sent_list = f.read().splitlines()

for sent in tqdm(sent_list):
    
    key = prefix + text_encode(sent)
    if r.exists(key):
        continue
    summary = retrieve_knowledge(sent, retrieve_type="summary")
    if not summary:
        continue
    sentences = sent_tokenize(summary)
    # 清洗
    for i, sentence in enumerate(sentences):
        if "References" in sentence:
            sentences = sentences[:i]
            break

    _topk = topk if len(sentences) > topk else len(sentences)
    sentences = [sent] + sentences

    with torch.no_grad():
        inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True).to(device)
        outputs = model(**inputs)

        embeddings = outputs.last_hidden_state[:, 0, :]
        # 第一句是输入句子，后面的是检索到的知识，做相似度计算
        cos_sim = F.cosine_similarity(embeddings[0:1], embeddings[1:], dim=1)

        # 计算topk高的相似度的句子
        topk_index = cos_sim.topk(_topk).indices
        topk_sentences = [sentences[i.item()] for i in topk_index]
        key = prefix + text_encode(sent)
        info = []
        for i, sentence in enumerate(topk_sentences):
            info.append((cos_sim[i].item(),sentence))
        r.set(key, json.dumps(info))

  0%|          | 2459/1000001 [02:21<15:58:40, 17.34it/s] 


KeyboardInterrupt: 